In [2]:
### Importar librerías
import yfinance as yf
import numpy as np
import pandas as pd

from datetime import datetime

### Datos del Sector Financiero

In [2]:
# Función para obtener datos de un índice de mercado o ETF usando la API de Yahoo Finance
def get_market_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        return data
    except Exception as e:
        print(f"Error getting data for {symbol} from Yahoo Finance: {e}")
        return None
    
# Obtener datos del mercado financiero desde el 01/01/2020 hasta hoy
start_date = "2024-06-01"
end_date = datetime.today().strftime("%Y-%m-%d")

# Utilizamos el Financial Select Sector SPDR Fund, que sigue el desempeño de las compañías en el 
# sector financiero del índice S&P 500.
financial_sector_symbol = "XLF"
# https://stockanalysis.com/etf/ → Si quisiéramos utilizar otros, habría que cambiar el símbolo

# Obtener datos para el sector financiero especificado desde Yahoo Finance
financial_sector_data = get_market_data(financial_sector_symbol, start_date, end_date)
financial_sector_data.drop(columns=['Adj Close'], inplace=True)
# HABLAR CON TXEMA Y MARÍA

# Guardar los datos en un archivo CSV
if financial_sector_data is not None:
    file_name = f"{financial_sector_symbol}_yahoo_finance_data.csv"
    financial_sector_data.to_csv(file_name)
    print(f"Data saved to {file_name}")
else:
    print("No data to save.")

csv_file = f"{financial_sector_symbol}_yahoo_finance_data.csv"
financial_sector_data = pd.read_csv(csv_file)

print(financial_sector_data)

[*********************100%%**********************]  1 of 1 completed

Data saved to XLF_yahoo_finance_data.csv
          Date       Open       High        Low      Close    Volume
0   2024-06-03  41.630001  41.680000  41.020000  41.380001  37855000
1   2024-06-04  41.160000  41.500000  41.000000  41.189999  27586400
2   2024-06-05  41.320000  41.360001  41.009998  41.310001  23817300
3   2024-06-06  41.380001  41.500000  41.130001  41.290001  39040200
4   2024-06-07  41.299999  41.709999  41.220001  41.480000  42273300
5   2024-06-10  41.299999  41.369999  41.049999  41.320000  31619000
6   2024-06-11  41.119999  41.139999  40.650002  40.849998  54584900
7   2024-06-12  41.270000  41.320000  40.750000  40.810001  46399800
8   2024-06-13  40.810001  40.820000  40.500000  40.779999  39497900
9   2024-06-14  40.500000  40.730000  40.369999  40.650002  29083000
10  2024-06-17  40.549999  41.020000  40.480000  41.009998  36076900
11  2024-06-18  40.959999  41.270000  40.919998  41.270000  38900300
12  2024-06-20  41.180000  41.599998  41.139999  41.490002  42

Estos precios están expresados en dólares estadounidenses.

### VIX & Historical Volatility

In [3]:
def get_vix_historical_data(start_date):
    # Símbolo del índice VIX en Yahoo Finance
    vix_ticker = "^VIX"
    # Definir las fechas de inicio y fin
    start_date = start_date
    end_date = datetime.today().strftime('%Y-%m-%d')
    # Obtener datos del VIX desde Yahoo Finance
    vix_data = yf.download(vix_ticker, start=start_date, end=end_date)
    return vix_data

if __name__ == "__main__":
    # Obtener datos históricos del VIX
    vix_historical_data = get_vix_historical_data()
    vix_historical_data.drop(columns=['Volume'], inplace=True)
    
    # Guardar datos en un archivo CSV
    file_name = "vix_data.csv"
    vix_historical_data.to_csv(file_name)
    print(f"Data saved to {file_name}")
    
csv_file_2 = "vix_data.csv"
vix_data = pd.read_csv(csv_file_2)

print("Historical VIX Data:")
print(vix_data)

[*********************100%%**********************]  1 of 1 completed

Data saved to vix_data.csv
Historical VIX Data:
            Date   Open       High    Low  Close  Adj Close
0     2020-01-02  13.46  13.720000  12.42  12.47      12.47
1     2020-01-03  15.01  16.200001  13.13  14.02      14.02
2     2020-01-06  15.45  16.389999  13.54  13.85      13.85
3     2020-01-07  13.84  14.460000  13.39  13.79      13.79
4     2020-01-08  15.16  15.240000  12.83  13.45      13.45
...          ...    ...        ...    ...    ...        ...
1123  2024-06-20  12.50  13.550000  12.18  13.28      13.28
1124  2024-06-21  13.22  13.780000  12.99  13.20      13.20
1125  2024-06-24  13.85  13.880000  13.15  13.33      13.33
1126  2024-06-25  13.48  13.520000  12.84  12.84      12.84
1127  2024-06-26  12.81  13.240000  12.37  12.55      12.55

[1128 rows x 6 columns]


In [4]:
def get_historical_volatility(symbol, start_date, end_date, window_size=30):
    try:
        # Descargar datos históricos desde Yahoo Finance
        data = yf.download(symbol, start=start_date, end=end_date)
        # Calcular los rendimientos logarítmicos
        data['Log Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
        # Calcular la volatilidad histórica
        data['Historical Volatility'] = data['Log Returns'].rolling(window=window_size).std() * np.sqrt(252) 
        # Volatilidad anualizada, es 252 porque es el numero de días que la bolsa está abierta
        
        data['Historical Volatility'] = data['Historical Volatility'] * 100  # Convertir a porcentaje
        data.reset_index(inplace=True)
        data = data[['Date', 'Adj Close', 'Historical Volatility']].dropna()

        return data
    except Exception as e:
        print(f"Error getting data for {symbol} from Yahoo Finance: {e}")
        return None

if __name__ == "__main__":
    # Definir el símbolo del índice
    symbol = "XLF" 
    start_date = "2020-01-01"
    end_date = datetime.today().strftime("%Y-%m-%d")
    window_size = 30  # Lapso de tiempo para el cálculo de volatilidad

    # Obtener datos de volatilidad histórica
    historical_volatility_data = get_historical_volatility(symbol, start_date, end_date, window_size)
    
    # Guardar los datos en un archivo CSV
    if historical_volatility_data is not None:
        file_name = f"{symbol}_historical_volatility.csv"
        historical_volatility_data.to_csv(file_name)  # Resetear el índice y guardar sin el índice
        print(f"Data saved to {file_name}")
    else:
        print("No data to save.")

    print(historical_volatility_data)

[*********************100%%**********************]  1 of 1 completed

Data saved to XLF_historical_volatility.csv
           Date  Adj Close  Historical Volatility
30   2020-02-14  28.476761              14.021854
31   2020-02-18  28.230093              13.901202
32   2020-02-19  28.440220              14.055945
33   2020-02-20  28.385403              13.920564
34   2020-02-21  28.111322              14.111807
...         ...        ...                    ...
1123 2024-06-20  41.326370              11.592704
1124 2024-06-21  41.167004              11.604835
1125 2024-06-24  41.590000              11.785484
1126 2024-06-25  41.279999              11.926762
1127 2024-06-26  41.020000              12.005836

[1098 rows x 3 columns]


El valor del VIX mide las expectativas del mercado sobre la volatilidad a corto plazo, calculada a partir de los precios de las opciones sobre el índice S&P 500. Por ejemplo, un valor del VIX de 13.46 el 02/01/2020 sugiere que el mercado espera una volatilidad anualizada del 13.46% durante el próximo mes. Esto es debido a que el VIX se expresa en términos anuales, es decir, que si las expectativas de volatilidad se mantuvieran constantes durante un año, se esperaría una desviación estándar de los rendimientos del 13.46%. Para convertir la volatilidad anualizada a una base mensual, se puede dividir por la raíz cuadrada de 12 (por 12 meses), lo que nos daría una volatilidad mensual del 3.88%, que indica que el mercado espera que el precio del S&P 500 podría moverse hacia arriba o hacia abajo aproximadamente un 3.88% durante el próximo mes.

### ALFA/BETA

Para el cálculo de alfa y beta, necesitaremos una tasa libre de riesgo. Debido a la naturaleza de nuestro análisis, utilizaremos el rendimiento del Tesoro de EE. UU. a 10 años, que es un punto de referencia comúnmente utilizado para la tasa libre de riesgo, especialmente para inversiones y análisis a largo plazo debido a su estabilidad.

In [5]:
def get_historical_prices(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close']

def calculate_daily_beta(stock_prices, market_prices, window_size=30):
    # Usamos 30 días para calcular la beta, lo que nos otorga fluctuaciones a más corto plazo.
    # Si quisiésemos enfocarnos más en el largo plazo, habría que aumentarlo.
    beta_values = []
    # Iterar 
    for i in range(len(stock_prices) - window_size + 1):
        stock_prices_window = stock_prices.iloc[i:i+window_size]
        market_prices_window = market_prices.iloc[i:i+window_size]
        
        stock_returns = stock_prices_window.pct_change().dropna()
        market_returns = market_prices_window.pct_change().dropna()
        
        # Calcula beta como la covarianza de los rendimientos de la acción con los del mercado dividida 
        # por la varianza de los rendimientos del mercado.
        covariance = stock_returns.cov(market_returns)
        market_variance = market_returns.var()
        
        beta = covariance / market_variance
        beta_values.append(beta)
    
    return pd.Series(beta_values, index=stock_prices.index[window_size-1:])

def calculate_daily_alpha(stock_prices, market_prices, risk_free_rate, window_size=30):
    alpha_values = []
    beta_values = calculate_daily_beta(stock_prices, market_prices, window_size)
    
    # Iterar
    for i in range(len(stock_prices) - window_size + 1):
        stock_prices_window = stock_prices.iloc[i:i+window_size]
        market_prices_window = market_prices.iloc[i:i+window_size]
        
        stock_returns = stock_prices_window.pct_change().dropna()
        market_returns = market_prices_window.pct_change().dropna()
        
        beta = beta_values.iloc[i]  # Obtener la beta para el lapso de tiempo establecido
        
        # Calculate market return (average daily return of market)
        market_return = market_returns.mean()
        
        # Calcular el rendimiento esperado utilizando CAPM
        expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
        
        # Rendimiento real de la acción
        actual_return = stock_returns.mean()
        
        # Calcular alfa
        alpha = actual_return - expected_return
        alpha_values.append(alpha)
    
    return pd.Series(alpha_values, index=stock_prices.index[window_size-1:])

def get_tresure_performance_10_years(start_date, end_date):
    # Símbolo del rendimiento del Tesoro a 10 años en Yahoo Finance
    tesoro_ticker = "^TNX"
    # Obtener datos históricos del rendimiento del Tesoro a 10 años
    treasure_performance = yf.download(tesoro_ticker, start=start_date, end=end_date)
    return treasure_performance['Adj Close']

def get_treasury_yield(symbol, start_date, end_date):
    # Descargar datos históricos para el rendimiento del Tesoro a 10 años
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close']

# Definir el símbolo para el rendimiento del Tesoro a 10 años
treasury_yield_symbol = "^TNX"
start_date = "2020-01-01"
end_date = "2024-06-15"

# Obtener rendimientos históricos
treasury_yield = get_treasury_yield(treasury_yield_symbol, start_date, end_date)

# Calcular la tasa libre de riesgo promedio
average_risk_free_rate = treasury_yield.mean() / 100  # Convert percentage to decimal

print(f"Average Risk-Free Rate from {start_date} to {end_date}: {average_risk_free_rate:.4f}")

# Ahora utilizaremos esta tasa libre de riesgo promedio en tus cálculos CAPM

if __name__ == "__main__":
    # Definir los tickers y lapso de tiempo
    xlf_symbol = 'XLF'  # Financial Select Sector SPDR Fund
    gspc_symbol = '^GSPC'  # S&P 500 Index
    start_date = '2020-01-01'
    end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Obtener precios históricos
    xlf_prices = get_historical_prices(xlf_symbol, start_date, end_date)
    gspc_prices = get_historical_prices(gspc_symbol, start_date, end_date)
    
    # Obtener el rendimiento histórico del Tesoro a 10 años y calcular la tasa libre de riesgo promedio
    treasure_performance = get_tresure_performance_10_years(start_date, end_date)
    risk_free_rate = treasure_performance.mean() / 100  # Convertir porcentaje a decimal
    
    # Calcular alfa y beta
    alpha_values = calculate_daily_alpha(xlf_prices, gspc_prices, risk_free_rate)
    beta_values = calculate_daily_beta(xlf_prices, gspc_prices)
    
    results = pd.DataFrame({
        'Date': alpha_values.index,
        'Alpha': alpha_values.values,
        'Beta': beta_values.values
    })
    
    print(results)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Average Risk-Free Rate from 2020-01-01 to 2024-06-15: 0.0251


[*********************100%%**********************]  1 of 1 completed


           Date     Alpha      Beta
0    2020-02-13  0.000897  1.084955
1    2020-02-14  0.000741  1.075097
2    2020-02-18  0.001118  1.092396
3    2020-02-19  0.001335  1.093210
4    2020-02-20  0.000979  1.076816
...         ...       ...       ...
1094 2024-06-20 -0.007827  0.738858
1095 2024-06-21 -0.008487  0.720294
1096 2024-06-24 -0.010324  0.628683
1097 2024-06-25 -0.011419  0.591082
1098 2024-06-26 -0.012137  0.574507

[1099 rows x 3 columns]


Beta mide qué tan sensible es un activo (en este caso, XLF) a los movimientos del mercado (S&P 500). 
- Una beta de 1 implica que el activo se mueve en línea con el mercado. 
- Una beta mayor que 1 indica una mayor volatilidad en comparación con el mercado. Puede indicar potenciales retornos más altos pero también un riesgo más elevado.
- Una beta menor que 1 indica una menor volatilidad en comparación con el mercado. Puede sugerir un riesgo más bajo pero posiblemente retornos más moderados.

Alfa mide diferencia entre el rendimiento real de un activo y su rendimiento esperado según el CAPM.
- Un alfa positivo indica que el activo ha tenido un desempeño mejor de lo esperado dado su nivel de riesgo (medido por la beta).
- Un alfa negativo indica que el activo ha tenido un desempeño peor de lo esperado dado su nivel de riesgo (beta).
- Un alfa cero indica que el activo ha rendido exactamente como se esperaba considerando su nivel de riesgo (beta).

In [6]:
# Convertir columnas de fecha a formato datetime
financial_sector_data['Date'] = pd.to_datetime(financial_sector_data['Date'])
vix_data['Date'] = pd.to_datetime(vix_data['Date'])
historical_volatility_data['Date'] = pd.to_datetime(historical_volatility_data['Date'])
results['Date'] = pd.to_datetime(results['Date'])

# Combinar DataFrames basados en la columna 'Date'
merged_data = pd.merge(financial_sector_data[['Date', 'Open', 'Close', 'Volume']],
                       vix_data[['Date', 'Open', 'Close']],
                       on='Date', how='inner')

merged_data = pd.merge(merged_data,
                       historical_volatility_data[['Date', 'Historical Volatility']],
                       on='Date', how='inner')

merged_data = pd.merge(merged_data,
                       results[['Date', 'Alpha', 'Beta']],
                       on='Date', how='inner')

merged_data.columns = [
    'Date', 
    'Financial_Sector_Open', 
    'Financial_Sector_Close', 
    'Volume', 
    'VIX_Open', 
    'VIX_Close', 
    'Historical_Volatility', 
    'Alpha', 
    'Beta'
]

print(merged_data)

         Date  Financial_Sector_Open  Financial_Sector_Close    Volume  \
0  2024-06-03              41.630001               41.380001  37855000   
1  2024-06-04              41.160000               41.189999  27586400   
2  2024-06-05              41.320000               41.310001  23817300   
3  2024-06-06              41.380001               41.290001  39040200   
4  2024-06-07              41.299999               41.480000  42273300   
5  2024-06-10              41.299999               41.320000  31619000   
6  2024-06-11              41.119999               40.849998  54584900   
7  2024-06-12              41.270000               40.810001  46399800   
8  2024-06-13              40.810001               40.779999  39497900   
9  2024-06-14              40.500000               40.650002  29083000   
10 2024-06-17              40.549999               41.009998  36076900   
11 2024-06-18              40.959999               41.270000  38900300   
12 2024-06-20              41.180000  

In [3]:
### 1.- Datos del Sector Financiero



# Función para obtener datos de un índice de mercado o ETF usando la API de Yahoo Finance
def get_market_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        return data
    except Exception as e:
        print(f"Error getting data for {symbol} from Yahoo Finance: {e}")
        return None
    
# Función para obtener VIX
def get_vix_historical_data(start_date):
    # Símbolo del índice VIX en Yahoo Finance
    vix_ticker = "^VIX"
    # Definir las fechas de inicio y fin
    start_date = start_date
    end_date = datetime.today().strftime('%Y-%m-%d')
    # Obtener datos del VIX desde Yahoo Finance
    vix_data = yf.download(vix_ticker, start=start_date, end=end_date)
    return vix_data    

# Función para obtener historical_volatility
def get_historical_volatility(symbol, start_date, end_date, window_size=30):
    try:
        # Descargar datos históricos desde Yahoo Finance
        data = yf.download(symbol, start=start_date, end=end_date)
        # Calcular los rendimientos logarítmicos
        data['Log Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
        # Calcular la volatilidad histórica
        data['Historical Volatility'] = data['Log Returns'].rolling(window=window_size).std() * np.sqrt(252) 
        # Volatilidad anualizada, es 252 porque es el numero de días que la bolsa está abierta
        data['Historical Volatility'] = data['Historical Volatility'] * 100  # Convertir a porcentaje
        data.reset_index(inplace=True)
        data = data[['Date', 'Adj Close', 'Historical Volatility']].dropna()
        return data        
    except Exception as e:
        print(f"Error getting data for {symbol} from Yahoo Finance: {e}")
        return None
    
# Funciones para la obtención de alfa y beta.    
def get_historical_prices(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close']

def calculate_daily_beta(stock_prices, market_prices, window_size=30):
    # Usamos 30 días para calcular la beta, lo que nos otorga fluctuaciones a más corto plazo.
    # Si quisiésemos enfocarnos más en el largo plazo, habría que aumentarlo.
    beta_values = []
    # Iterar 
    for i in range(len(stock_prices) - window_size + 1):
        stock_prices_window = stock_prices.iloc[i:i+window_size]
        market_prices_window = market_prices.iloc[i:i+window_size]            
        stock_returns = stock_prices_window.pct_change().dropna()
        market_returns = market_prices_window.pct_change().dropna()
        
        # Calcula beta como la covarianza de los rendimientos de la acción con los del mercado dividida 
        # por la varianza de los rendimientos del mercado.
        covariance = stock_returns.cov(market_returns)
        market_variance = market_returns.var()            
        beta = covariance / market_variance
        beta_values.append(beta)        
    return pd.Series(beta_values, index=stock_prices.index[window_size-1:])
def calculate_daily_alpha(stock_prices, market_prices, risk_free_rate, window_size=30):
    alpha_values = []
    beta_values = calculate_daily_beta(stock_prices, market_prices, window_size)        
    # Iterar
    for i in range(len(stock_prices) - window_size + 1):
        stock_prices_window = stock_prices.iloc[i:i+window_size]
        market_prices_window = market_prices.iloc[i:i+window_size]            
        stock_returns = stock_prices_window.pct_change().dropna()
        market_returns = market_prices_window.pct_change().dropna()            
        beta = beta_values.iloc[i]  # Obtener la beta para el lapso de tiempo establecido            
        # Calculate market return (average daily return of market)
        market_return = market_returns.mean()            
        # Calcular el rendimiento esperado utilizando CAPM
        expected_return = risk_free_rate + beta * (market_return - risk_free_rate)            
        # Rendimiento real de la acción
        actual_return = stock_returns.mean()            
        # Calcular alfa
        alpha = actual_return - expected_return
        alpha_values.append(alpha)        
    return pd.Series(alpha_values, index=stock_prices.index[window_size-1:])
def get_tresure_performance_10_years(start_date, end_date):
    # Símbolo del rendimiento del Tesoro a 10 años en Yahoo Finance
    tesoro_ticker = "^TNX"
    # Obtener datos históricos del rendimiento del Tesoro a 10 años
    treasure_performance = yf.download(tesoro_ticker, start=start_date, end=end_date)
    return treasure_performance['Adj Close']
def get_treasury_yield(symbol, start_date, end_date):
    # Descargar datos históricos para el rendimiento del Tesoro a 10 años
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close']

def format(df):
    if df is not None: 
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)         
        df['Date'] = pd.to_datetime(df['Date'])
    return df


start_date = '2024-06-10'
end_date = datetime.today().strftime("%Y-%m-%d")

# 1.1.- Obtener datos del Sector Financiero desde Yahoo Finance
financial_sector_symbol = "XLF"
financial_sector_data = get_market_data(financial_sector_symbol, start_date, end_date)
if financial_sector_data is not None: 
    financial_sector_data.drop(columns=['Adj Close'], inplace=True)
financial_sector_data = format(financial_sector_data)

# 1.2.- Obtener datos históricos del VIX
vix_data = get_vix_historical_data(start_date)
if vix_data is not None: 
    vix_data.drop(columns=['Volume'], inplace=True)
vix_data = format(vix_data)

# 1.3.- Obtener datos de volatilidad histórica
symbol = "XLF" 
window_size = 30  # Lapso de tiempo para el cálculo de volatilidad
new_start_date = datetime.strptime(start_date, '%Y-%m-%d')-timedelta(days=1.5*window_size)
new_start_date = new_start_date.strftime("%Y-%m-%d")
historical_volatility_data = get_historical_volatility(symbol, new_start_date, end_date, window_size)
historical_volatility_data['Date'] = pd.to_datetime(historical_volatility_data['Date'])

# 1.4.- Obtención de alfa y beta.
    # Obtener rendimientos históricos
treasury_yield_symbol = "^TNX"
treasury_yield = get_treasury_yield(treasury_yield_symbol, new_start_date, end_date)
if treasury_yield is not None:
    average_risk_free_rate = treasury_yield.mean() / 100  # Convert percentage to decimal
xlf_symbol = 'XLF'  # Financial Select Sector SPDR Fund
gspc_symbol = '^GSPC'  # S&P 500 Index

    # Obtener precios históricos
xlf_prices = get_historical_prices(xlf_symbol, new_start_date, end_date)
gspc_prices = get_historical_prices(gspc_symbol, new_start_date, end_date)

    # Obtener el rendimiento histórico del Tesoro a 10 años y calcular la tasa libre de riesgo promedio
treasure_performance = get_tresure_performance_10_years(new_start_date, end_date)
if treasure_performance is not None:
        risk_free_rate = treasure_performance.mean() / 100  # Convertir porcentaje a decimal

    # Calcular alfa y beta
alpha_values = calculate_daily_alpha(xlf_prices, gspc_prices, risk_free_rate)
beta_values = calculate_daily_beta(xlf_prices, gspc_prices)

results = pd.DataFrame({
    'Date': alpha_values.index,
    'Alpha': alpha_values.values,
    'Beta': beta_values.values  })
results['Date'] = pd.to_datetime(results['Date'])


    
# Combinar DataFrames basados en la columna 'Date'
if financial_sector_data is not None and vix_data is not None:  
    merged_data = pd.merge(financial_sector_data[['Date', 'Open', 'Close', 'Volume']],
                           vix_data[['Date', 'Open', 'Close']],
                           on='Date', how='inner')
if merged_data is not None and historical_volatility_data is not None:  
    merged_data = pd.merge(merged_data,
                           historical_volatility_data[['Date', 'Historical Volatility']],
                           on='Date', how='inner')
if merged_data is not None and results is not None:  
    merged_data = pd.merge(merged_data,
                           results[['Date', 'Alpha', 'Beta']],
                           on='Date', how='inner')
# Combinar DataFrames basados en la columna 'Date'
merged_data.columns = [
    'Date', 
    'Financial_Sector_Open', 
    'Financial_Sector_Close', 
    'Volume', 
    'VIX_Open', 
    'VIX_Close', 
    'Historical_Volatility', 
    'Alpha', 
    'Beta'
]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
